<h1>Cargado de Datasets a BB.DD</h1>

<h2>Sinopsis del programa</h2>
<h4>Hare lecturas de diferentes archivos csv y json sobre videojuegos en stream, recomendaciones hacia los juegos y usuarios que los jugaron o escribieron una recomendacion<h4>
<p><b>Indices</b></p>
<ul>
    <li><b>Lectura de archivos</b></li>
    <li><b>Conexion a BB.DD</b></li>
    <li><b>Definicion de funciones</b></li>
    <li><b>Cargado en BB.DD con funciones</b></li>
    <li><b>Queris a la BB.DD</b></li>
</ul>

<h3><u>Imports</u></h3>

In [1]:
# Imports necesarios para la lectura, tratado y generado de queris
import pandas as pd
import mysql.connector

<h2><u>Funciones con las que trabajaremos</u></h2>

<h3><u>Funcion de cargado de archivos</u></h3>

In [2]:
def carga():
    # Lectura de archivo
    games1 = pd.read_csv("Datasets/games.csv")
    # DataFrame acotado
    games2 = games1.head(2000)
    # Lectura de archivo
    users1 = pd.read_json("Datasets/users.json")
    # DataFrame acotado
    users2 = users1.head(100000)
    # Lectura de archivo
    recom1 = pd.read_csv("Datasets/recommendations.csv")
    # DataFrame acotado
    recom2 = recom1.head(100000)
    #return con todos los dataframes
    return games1 , games2 ,users1 , users2 , recom1 , recom2
    

<h3><u>Funcion que genera el objeto de conexion entre python y mysql</u></h3>

In [3]:
# Aqui crearemos un objeto de conexion que servira como puente entre nuestro programa y la base de datos que definamos 
# Este objeto es creado utilizando la libreria mysql.connector de mysql
# Haciendo un try except nos aseguramos de manejar las excepciones que ocasionen errores devolviendo informacion mas detallada
def connect(contraseña):
    try:
        # Generamos el objeto que creara la conexion con los parametros de nuestra base
        db = mysql.connector.connect(user='root',
                                    password=f'{contraseña}',
                                    host='localhost',
                                    database='steam')
        return db
    # Crearemos excepciones para diferentes casos como:
    except mysql.connector.Error as error:
        
        # Error de acceso denegado a la base de datos por usuaio 
        if error.errno == mysql.connector.errorcode.ER_ACCESS_DENIED_ERROR:
            print("Error: Acceso denegado para el usuario.")
        # Error de intento de acceso a una base de datos inexistente
        elif error.errno == mysql.connector.errorcode.ER_BAD_DB_ERROR:
            print("Error: La base de datos no existe.")
        # Error de contraseña incorrecta
        elif error.sqlstate == '28000':
            print("Error: Incorrect password.")
        # Error de conexion generico
        else:
            print(f"Error conectando a la base de datos: {error}")     
        return db


<h3><u>Funciones de cargado de datos a MySQL</u></h3>

In [4]:
# La funcion to string pasa una lista de elementos a unn string separado por comas con todos esos elementos
def to_string(cols):
    # El return devuelve el join de la lista con ", " ente elementos
    return ", ".join(cols)
# La funcion valorise toma como entrada una lista y devuelve una lista de igual tamaño en la que todos los elementos son "%s"
def valorise(cols):
    # Generamos una nueva variable llamada col que es la multiplicacion por la len de cols de la lista [%s]
    # En python se pueden multiplicar listas generando listas más grandes con el mismo patron de elementos
    col = ['%s']*len(cols)
    # Devolvemos la lista nueva
    return col
# La funcion to_mysql toma como entrada un dataframe y un nombre de tabla y pasa directamente un dataframe entero a la BB.DD   
def to_mysql(data,table):
    # Generamos un cursor para poder interactuar con la BB.DD
    curs = db.cursor()
    # Seteamos la validacion del constraint de foreign key a false dado que en una tabla referenciada hay menos datos que 
    # en la tabla que referencia para evitar erroresd de inserrcion
    curs.execute("SET FOREIGN_KEY_CHECKS = 0;")
    # Sacamos una lista con todos los valores de las columnas del DataFrame
    cols = data.columns.values.tolist()
    # Iteramos en el dataframe con iterrows fila por fila
    for _, row in data.iterrows():
        # Hacemos una lista con todos los elementos de la fila
        val = row.to_list()
        # Creamos un string con un insert de sql con el nombre de la tabla en la que queremos insertar los datos
        # Las columnas a las que queremos afectar y los tipos de valores a sustituir
        sql = f"INSERT INTO {table} ({to_string(cols)}) VALUES ({to_string(valorise(cols))})"
        # Un try except intentara introducir los valores a ser posible
        try:
            # El execute genera la queri con el sql y el val
            curs.execute(sql, val)
		# Generaremos excepciones tales que si la consulta esta mal nos explque porque
        except Exception as e:
            # Nos dice cual ha sido el error
            print(f"Error: {str(e)}")
            # En caso de error, la funcion rollback eliminara todos los cambios que se hayan hecho a la BB.DD
            db.rollback()
            return
    # Haremos un print declarando que la inserccion ha funcionado
    print("La inserccion ha sido exitosa")
    # Volvemos a setear el checkeo a True
    curs.execute("SET FOREIGN_KEY_CHECKS = 1;")
    # Haremos un comit para que los cambios se pasen al puente y por ende a la BB.DD
    db.commit()
    # Cerraremos el cursor
    curs.close()


<h3><u>Funciones de Selection</u></h3>

In [5]:
def get_columns(string):
    # Declaramos el inicio de las columnas + 6 dado que select tiene 6 caracteres
    ini = string.find("SELECT") + 6
    # Declaramos el final dado que solo nos interesan las columnas antes de from
    fin = string.find("FROM")
    # Hacemos un srip de la parte del string que nos interesa
    cols = string[ini:fin].strip()
    # Le hacemos un split separado por comas para meterlo a una lista
    cols = cols.split(",")
    # Bucle que recorre toda la lista
    # Le quitamos cualquier espacio que pueda tener elemento
    cols = [x.strip(' ') for x in cols]
    # Devolvemos la lista
    return cols
def get_table(string):
    # Declaramos el inicio de las columnas + 6 dado que select tiene 6 caracteres
    ini = string.find("FROM") + 4
    # Declaramos el final dado que solo nos interesan las columnas antes de from
    fin = ini + 6
    # Hacemos un srip de la parte del string que nos interesa
    string = string[ini:fin].strip()
    # Devuelvo la tabla
    return string

def get_cols_s(string):
    # Llamamos a la funcion get table que nos dara el nombre de la tabla afectada
    string = get_table(string)
    # Crearemos un string con una queri que nos devolvera informacion sobre las columnas como nombre typo ,etc
    sql = "SHOW COLUMNS FROM {}".format(string)
    # Creamos un cursor
    curs = db.cursor()
    #Enseñaremos las columnas de la tabla
    curs.execute(sql)
    # Hacemos una lista con los nombres
    cols = [col[0] for col in curs.fetchall()]
    # Cerramos el cursor
    curs.close()
    #Devolvemos la lista
    return cols
    
def selection_query(string):
    # Creearemos un cursor
    curs = db.cursor()
    # Ejecutaremos la query
    curs.execute(string)
    # Extraeremos todo los datos del cursor
    res = curs.fetchall()
    # Cerraremos el cursor
    curs.close()
    # Cogeremos las columnas de la query
    cols = get_columns(string)
    # Los meteremos a un dataframe
    # Si pide todas las columnas habra que hacerlo de forma diferente para que quede elegante
    if(cols[0] != "*"):
        res = pd.DataFrame(data = res, columns =cols )
    else:
        res = pd.DataFrame(data = res, columns = get_cols_s(string))
    #Devolvemos el dataframe
    return res

<h3><u>Funciones de Update</u></h3>

In [6]:
def get_sql_u(string):
     # Declaramos el inicio de las columnas + 6 dado que select tiene 6 caracteres
    ini = string.find("UPDATE") + 6
    # Declaramos el final dado que solo nos interesan las columnas antes de from
    fin = ini + 6
    # Hacemos un srip de la parte del string que nos interesa
    string = string[ini:fin].strip()
    sql = "SELECT * FROM {}".format(string)
    # Devuelve el sql del queri
    return sql
def get_new_cols_u(string):
    # Declaramos el inicio de las columnas + 6 dado que select tiene 6 caracteres
    ini = string.find("UPDATE") + 6
    # Declaramos el final dado que solo nos interesan las columnas antes de from
    fin = ini + 6
    # Hacemos un srip de la parte del string que nos interesa
    string = string[ini:fin].strip()
    # Crearemos un string con una queri que nos devolvera informacion sobre las columnas como nombre typo ,etc
    sql = "SHOW COLUMNS FROM {}".format(string)
    # Creamos un cursor
    curs = db.cursor()
    #Enseñaremos las columnas de la tabla
    curs.execute(sql)
    # Hacemos una lista con los nombres
    cols = [col[0] for col in curs.fetchall()]
    # Cerramos el cursor
    curs.close()
    #Devolvemos la lista
    return cols
    
def update_query(string):
    # Creearemos un cursor
    curs = db.cursor()
    # Ejecutaremos la queri
    curs.execute(string)
    curs.execute(get_sql_u(string))
    # Extraeremos todo los datos del cursor
    res = curs.fetchall()
    # Metemos la tabla a un dataframe
    try:
        res = pd.DataFrame(data = res, columns = get_new_cols_u(string))
    except:
        res = pd.DataFrame(data = res)
    # Cerraremos el cursor
    curs.close()
    
    return res

<h3><u>Funciones de Alter</u></h3>

In [7]:
def get_sql_a(string):
     # Declaramos el inicio de las columnas + 6 dado que select tiene 6 caracteres
    ini = string.find("TABLE") + 5
    # Declaramos el final dado que solo nos interesan las columnas antes de from
    fin = ini + 6
    # Hacemos un srip de la parte del string que nos interesa
    string = string[ini:fin].strip()
    sql = "SELECT * FROM {}".format(string)
    # Devuelve el sql del queri
    return sql

def get_new_cols_a(string):
    # Declaramos el inicio de las columnas + 6 dado que select tiene 6 caracteres
    ini = string.find("TABLE") + 5
    # Declaramos el final dado que solo nos interesan las columnas antes de from
    fin = ini + 6
    # Hacemos un srip de la parte del string que nos interesa
    string = string[ini:fin].strip()
    # Crearemos un string con una queri que nos devolvera informacion sobre las columnas como nombre typo ,etc
    sql = sql = "SHOW COLUMNS FROM {}".format(string)
    # Creamos un cursor
    curs = db.cursor()
    #Enseñaremos las columnas de la tabla
    curs.execute(sql)
    # Hacemos una lista con los nombres
    cols = [col[0] for col in curs.fetchall()]
    # Cerramos el cursor
    curs.close()
    #Devolvemos la list
    return cols

def alter_query(string):
    # Creearemos un cursor
    curs = db.cursor()
    # Le decimos al cursosr que ejecute la orden de no chequear Foreign keys para que no de problemas de borrado
    curs.execute("SET FOREIGN_KEY_CHECKS = 0;")
    # Ejecutaremos la queri
    curs.execute(string)
    curs.execute(get_sql_a(string))
    # Extraeremos todo los datos del cursor
    res = curs.fetchall()
    # Metemos la tabla a un dataframe
    try:
        res = pd.DataFrame(data = res, columns = get_new_cols_a(string))
    except:
        res = pd.DataFrame(data = res)
    # Lo volveremos a reactivar
    curs.execute("SET FOREIGN_KEY_CHECKS = 1;")
    # Cerraremos el cursor
    curs.close()
    return res

<h3><u>Funciones de Borrado</u></h3>

In [8]:
def get_sql_b(string):
     # Declaramos el inicio de las columnas + 6 dado que select tiene 6 caracteres
    ini = string.find("FROM") + 4
    # Declaramos el final dado que solo nos interesan las columnas antes de from
    fin = ini + 6
    # Hacemos un srip de la parte del string que nos interesa
    string = string[ini:fin].strip()
    sql = "SELECT * FROM {}".format(string)
    # Devuelve el sql del queri
    return sql

def get_new_cols_b(string):
    # Declaramos el inicio de las columnas + 6 dado que select tiene 6 caracteres
    ini = string.find("FROM") + 4
    # Declaramos el final dado que solo nos interesan las columnas antes de from
    fin = ini + 6
    # Hacemos un srip de la parte del string que nos interesa
    string = string[ini:fin].strip()
    # Crearemos un string con una queri que nos devolvera informacion sobre las columnas como nombre typo ,etc
    sql = sql = "SHOW COLUMNS FROM {}".format(string)
    # Creamos un cursor
    curs = db.cursor()
    #Enseñaremos las columnas de la tabla
    curs.execute(sql)
    # Hacemos una lista con los nombres
    cols = [col[0] for col in curs.fetchall()]
    # Cerramos el cursor
    curs.close()
    #Devolvemos la list
    return cols

def errase_query(string):
    # Creearemos un cursor
    curs = db.cursor()
    # Le decimos al cursosr que ejecute la orden de no chequear Foreign keys para que no de problemas de borrado
    curs.execute("SET FOREIGN_KEY_CHECKS = 0;")
    # Ejecutaremos la query
    curs.execute(string)
    curs.execute(get_sql_b(string))
    # Extraeremos todo los datos del cursor
    res = curs.fetchall()
    # Metemos la tabla a un dataframe
    try:
        res = pd.DataFrame(data = res, columns = get_new_cols_b(string))
    except:
        res = pd.DataFrame(data = res)
    # Volvemos a activar el chequeo
    curs.execute("SET FOREIGN_KEY_CHECKS = 1;")
    # Cerraremos el cursor
    curs.close()
    return res

<h2><u>Lectura de Archivos</u></h2>

<p><b>Llamarremos a la funcion 'carga' que cargara todos nuestros archivos a dataframes y luego veremos sus columnas con head</b></p>

In [9]:
games1 , games2 ,users1 , users2 , recom1 , recom2 = carga()

In [10]:
games1.columns

Index(['app_id', 'title', 'date_release', 'win', 'mac', 'linux', 'rating',
       'positive_ratio', 'user_reviews', 'price_final', 'price_original',
       'discount', 'steam_deck'],
      dtype='object')

In [11]:
users1.head()

,user_id,products,reviews
0,4154727,156,1
1,2637640,329,2
2,1438551,176,1
3,1549343,98,2
4,1712796,144,2


In [12]:
recom1.columns

Index(['app_id', 'helpful', 'funny', 'date_', 'is_recommended', 'hours',
       'user_id', 'review_id'],
      dtype='object')

<h3> Ya definidas y terminadas solo queda llamar a la funcion to_mysql e insertarle los parametros de todos los dataframes</h3>
<p><b>En esta parte de la practica dado que metemos Dataframes enteros a la BB.DD concluimos con el apartado de Querys de inserccion a la BB.DD dado que la carga de los dataframes no es mas que la inserccion de todas sus rows a las tablas:
    <ul>
        <li>Carga y Grabacion de DataFrames directamente a  BB.DD</li>
    </ul>
<i>Ha de notarse que al no estar usando slqalchemy (sqlite) sino el conector propio de mysql no hay funciones como df.to_sql y debe sermanualmente realizada la inserccion</i></b></p>

<h3><u>Conexion a BB.DD</u></h3>

In [14]:
# Si vas a ejecutar el programa y vas a conectar con mysql deberas poner tu propia contraseña dado que guardaras el sql en tu local host
db = connect(input())

<h3><u>Cargado en la BB.DD con las funciones</u></h3>

In [15]:
to_mysql(games1,"games")   

La inserccion ha sido exitosa


In [16]:
to_mysql(users2,"users")

La inserccion ha sido exitosa


In [17]:
to_mysql(recom2,"recom")

La inserccion ha sido exitosa


In [ ]:
db.close()

<h3><u>Consultas a la BB.DD</u><h3>
<p><b>Una vez ya cargados los datos en la BB.DD podemos empezar a hacer consultas a esta misma</p></b>
Haremos consultas del tipo:
<ul>
	<li><b>Seleccion</b></li>
    <ul>
   		<li><b>Seleccion Cruzada</b></li>
        <li><b>Consulas del tipo que hay en una tabla que no hay en otra</b></li>
    </ul>
 	<li><b>Actulizacion</b></li>
 	<li><b>Insercion</b></li>
 	<li><b>Borrado de datos de manera directa</b></li>
</ul>
</b></p>

<h3><u>Seleciones</u></h3>

<p><b>Seleccionaremos toda la inormacion de las primeras 10000 filaes de la tabla Users</b></p>

In [ ]:
selection_query("SELECT * FROM users LIMIT 1000").head()

,user_id,products,reviews,help
0,201845,867,5,SI
1,670866,304,7,SI
2,1172766,165,4,SI
3,1659984,1773,4,SI
4,2031886,298,5,SI


<p><b>Seleccionare informacion relevante para un consumidor interesado en un juego compatible con Windows, Mac y Linux de la tabla Games</b></p>

In [ ]:
selection_query("SELECT title, date_release, positive_ratio, user_reviews, price_final FROM games WHERE win = 1 AND mac = 0 AND linux = 1 GROUP BY positive_ratio ORDER BY price_final").head()

ProgrammingError: 1054 (42S22): Unknown column 'win' in 'where clause'

<p><b>Seleccionamos informacion entre dos tablas recom y users para ver usuarios que escribieron reseñas sobre un juego, cuantos juegos tiene el usuario, cuantas reseñas ha hecho, cuan buena o divertida fue la recomendacion y cuanto tiempo jugo al juego antes de hacer ñla recomendacion. Con esto podemos filtras de buena manera cuan rigurosaas o buenas son las recomendaciones dependiendo de muchos factores </b></p>

In [ ]:
selection_query("SELECT users.user_id, users.products, users.reviews, recom.review_id, recom.hours, recom.helpful, recom.funny FROM users JOIN recom ON users.user_id = recom.user_id WHERE users.user_id = recom.user_id GROUP BY users.reviews ORDER BY recom.helpful").head()

<p><b>Selecionaremos informacion de la tabla de recomendaciones en las que hayan sido escritas por un usuario registrado en 
   nuestros datasets</b></p>

In [ ]:
selection_query("SELECT recom.user_id , review_id , helpful , funny FROM recom RIGHT JOIN users ON recom.user_id = users.user_id GROUP BY recom.user_id;").head()

<p><b>Seleccionaremos todos los juegos de games que no tengan una review en recom</b></p>

In [ ]:
selection_query("SELECT games.title, games.app_id FROM games WHERE games.app_id NOT IN (SELECT recom.app_id FROM recom) ORDER BY games.title").head()

<h3><u>Actualización</u></h3>

<p><b>Actualizaremos la tabla recom para que donde funny sea > que 5 lo buena que sea la respuesta se ponga automaticamente en 100 </b></p>

In [ ]:
update_query("UPDATE recom SET helpful = 100 WHERE funny > 5").head()

<p><b>Actializaremos la tabla para que si una review tiene 300 horas o mas de juego is_recomendes sea igual a 5</b></p>

In [ ]:
update_query("UPDATE recom SET is_recommended = 5 WHERE hours >= 300").head()

<p><b>Hare un update a users creando una nueva columna que pondra "si" si una review que el usuario haya hecho ha sido muy util o helpful >= 10, y pondra no si no lo ha sido</b></p>

In [ ]:
alter_query("ALTER TABLE users ADD column help varchar(2)")

In [ ]:
update_query("UPDATE users SET help = (CASE WHEN user_id IN ( SELECT user_id FROM recom WHERE helpful >= 11 AND hours >20) THEN 'SI' ELSE 'NO' END);").head()

<h3><u>Borrado de datos de manera directa</u></h3>

<p><b>Eliminado de partes de la tabla de usiarios donde su contribucion no ha sido suficientemente util es decir help = no o que no tienen reviews suficientes o reviews < 4 </b></p>

In [ ]:
errase_query("DELETE FROM users WHERE (help = 'no' OR reviews < 4)"  )

<p><b>Eliminacion de columnas enteras de sistemas soportados para las apps de la tabla games</b></p>

In [ ]:
alter_query("ALTER TABLE games DROP COLUMN win, DROP COLUMN mac, DROP COLUMN linux;").head()